### Векторизация слов из нашего датасета

In [30]:
import spacy
import csv
import pandas as pd
#from pymystem3 import Mystem
import re
import os
import numpy as np
import gensim

In [2]:
#это я для себя, чтобы пакеты точно работали, вообще не нужно
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [4]:
df = pd.read_csv('questions_answers_titles.csv',sep = ';',usecols=['text_question', 'text_answer', 'text_title'])
df[:10]

,text_question,text_answer,text_title
0,"Играя онлайн, какими средствами ввода-вывода и...",Зависит от количества контор в которых вы соби...,Ввод-вывод средств
1,Сколько вам платят ВХ и Сбо за пятерки в рейти...,"Вот <a href=""https://bookmaker-ratings.ru/rasp...",Сколько стоит пятерка на РБ?
2,"Вам нужны сотрудники? Опыт - два года на ""Чемп...","Добрый день, Максим!\n\nОтправьте свое резюме ...",Работа
3,Считается ли вилкой ставка до матча на событие...,"Разумеется, что нет. Во-первых, в live, во все...",Вилки
4,"Здравствуйте, читал пост о порезке счетов, воз...",Можно выигрывать если ставить например провере...,Сколько можно выигрывать
5,"Я правильно понимаю:-,если меня спалила контор...","Мультиаккаунтинг нельзя доказать, поэтому плат...",вопрос по сути:-)
6,"В ответе на мою претензию указанно, что я отно...","Дайте ссылку на вашу жалобу пожалуйста, я посм...",претензия
7,Должны ли конторы давать возвраты по договорны...,Согласен с самым первым комментарием. Дело в т...,Договорные матчи
8,В каких конторах возможен вывод средств через ...,Эта информация есть на нашем сайте!<br />\nhtt...,Western Union
9,"Бк мачбук заблокировала мой счет, помогите раз...",Заполните пожалуйста форму<br />\nhttps://book...,помогите


небольшой препроцессинг (нужно лучше чистить)

In [5]:
for name in df.columns.tolist():
    df[name].replace(to_replace=r"[a-zA-Z\n\r\\\*\#\%\d+\/\@\>\<\(\)\$\_\€]", value=r"", regex=True, inplace=True)
    df[name].replace(to_replace=r"[?\!\,\.\;\:\«\»\'\"\-\=]", value=r" ", regex=True, inplace=True)
    df[name].replace(to_replace=r"[\s+]", value=r" ", regex=True, inplace=True)

In [6]:
df.to_csv('clean_df.csv', sep=';',encoding='utf-8', index = False,header=False)

In [7]:
with open('cleantxt.txt', "w",encoding="utf8") as my_output_file:
    with open('clean_df.csv', "r",encoding="utf8") as my_input_file:
        [ my_output_file.write("".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

#### Проверяем вхождение слов в словаре, если не входят, то мы их векторизуем

In [8]:
def slurp(path):
    with open(path, 'r', encoding = 'utf-8') as file_object:
        return file_object.read()

In [9]:
rucorpora_dict = slurp('dict.opcorpora.txt')

In [10]:
rucorpora_dict = re.findall(r'[А-ЯЁ]+', rucorpora_dict)
rucorpora_dict_low = [i.lower() for i in rucorpora_dict]

In [11]:
with open('cleantxt.txt', "r",encoding="utf-8") as my_file:
    texts = [line.lower().split(' ') for line in my_file]

In [12]:
def list_out_list(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all

In [13]:
texts_all = list_out_list(texts)

итоговый список слов для векторизации

In [56]:
result=list(set(texts_all) & set(rucorpora_dict_low))
len(result)

12699

In [15]:
with open('list_of_words.txt', "w",encoding="utf8") as file:
    for row in result:
        file.write(row +'\n')

#### Делаем векторы для нашего списка слов

In [16]:
#это работает довольно долго
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('cc.ru.300.bin')

In [17]:
vects=[]
errors=[]
for word in result:
    try:
        v = model[word]
        vects.append(v)
    except KeyError:
        print('I got a KeyError - reason "%s"' % str(word))
        errors.append(word)

I got a KeyError - reason "гуам"
I got a KeyError - reason "умео"
I got a KeyError - reason "аик"
I got a KeyError - reason "арн"


In [54]:
print(len(vects)) #сколько у нас слов векторизовано
print(len(result)) #сколько у нас слов было до векторизации (с теми, для которых нет nграмм)

12695
12699


In [40]:
errors

['гуам', 'умео', 'аик', 'арн']

In [58]:
for word in result:
    for item in errors:
        if item==word:
            result.remove(word)

In [61]:
len(result) #после удаления невекторизованных слов

12695

In [48]:
#сохраним сначала массивы векторов
with open('vectors.txt', "w",encoding="utf8") as output:
            np.savetxt(output, vects, newline='\n')

In [64]:
#запишем к ним слова, которые векторизовались, чтобы было как в оригинальном файле cc.ru.300.vec
with open('vectors.txt', "r",encoding="utf8") as inputfile:
    with open('vectors2.txt', "w",encoding="utf8") as outputfile:
        a = inputfile.readline()
        for word in result:
            outputfile.write(word+' '+a)
    

#### Дописываем наши векторы к cc.ru.300.vec

In [ ]:
with open('cc.ru.300copy.vec', 'a', encoding="utf8") as bigfile:
    with open('vectors2.txt', "r",encoding="utf8") as smallfile:
        our_vects = smallfile.read()
        bigfile.write(our_vects)

In [ ]:
# изменяем первую строку на актуальное количество слов
with open('cc.ru.300copy.vec', 'r+', encoding="utf-8") as f:
    line = next(f)
    count = int(line[0:7])
    string = str(count + len(result))
    old = str(count)+' 300\n'
    new = string + ' 300\n' 
    f.seek(0) 
    f.write(line.replace(old, new))